一、
下载movielen-latest

二、
根据movielen的数据内容以及Personalize导入数据的格式要求，编写Avro Json作为数据的Schema。
```json
{"type": "record", 
"name": "Interactions", 
"namespace": "com.amazonaws.personalize.schema",
"fields":[
    {"name": "ITEM_ID", "type": "string"},
    {"name": "USER_ID", "type": "string"},
    {"name": "TIMESTAMP", "type": "long"}
],
"version": "1.0"}
```
将Schema的Json导入到Personalize

三、
简单处理，认为rating>=4为用户感兴趣的标志，因此仅将rating>=4的数据用于训练。
由于用全量数据会导致训练时间过久，因此demo中只取其中10w条记录。

```python
import pandas, boto3 
from sklearn.utils import shuffle
ratings = pandas.read_csv('ratings.csv')
ratings = shuffle(ratings)
ratings = ratings[ratings['rating']>=4.0]
ratings = ratings.drop(columns='rating')
ratings.columns = ['USER_ID','ITEM_ID','TIMESTAMP']
ratings = ratings[:100000]
ratings.to_csv('ratings.processed.csv',index=False)
```
然后将文件导入到Personalize

四、
选择AutoML的方式，将训练算法和参数交给Personalize根据上传的数据自行判断。

五、
在Personalize上在Solution的基础上创建Campaign

六、
验证Campaign API的推荐效果

加载movielens数据，创建movie id到movie内容的映射函数。
切换到已经能preview Personalize服务的账号上。

In [1]:
import pandas as pd
import boto3

mov = pd.read_csv('movies.csv')

def movid2title(id):
    return mov[mov.movieId == int(id)].title.values[0]

def movid2genres(id):
    return mov[mov.movieId == int(id)].genres.values[0]

# Switch to test profile to run personalize
boto3.setup_default_session(profile_name='bbkuser')

调接口根据用户id推荐结果

In [2]:
personalize = boto3.client('personalize-runtime')
USER_ID = '333'

response = personalize.get_recommendations(
    campaignArn = "arn:aws:personalize:us-west-2:188869792837:campaign/personalize-demo2-camp",
    userId = USER_ID)

print("Recommended items")
for item in response['itemList']:
    print (movid2title(item['itemId']), movid2genres(item['itemId']))

Recommended items
('Shawshank Redemption, The (1994)', 'Crime|Drama')
('Forrest Gump (1994)', 'Comedy|Drama|Romance|War')
('Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller')
('Silence of the Lambs, The (1991)', 'Crime|Horror|Thriller')
('Matrix, The (1999)', 'Action|Sci-Fi|Thriller')
('Fight Club (1999)', 'Action|Crime|Drama|Thriller')
('Star Wars: Episode IV - A New Hope (1977)', 'Action|Adventure|Sci-Fi')
('Godfather, The (1972)', 'Crime|Drama')
("Schindler's List (1993)", 'Drama|War')
('Star Wars: Episode V - The Empire Strikes Back (1980)', 'Action|Adventure|Sci-Fi')
('Usual Suspects, The (1995)', 'Crime|Mystery|Thriller')
('Braveheart (1995)', 'Action|Drama|War')
('Lord of the Rings: The Two Towers, The (2002)', 'Adventure|Fantasy')
('Jurassic Park (1993)', 'Action|Adventure|Sci-Fi|Thriller')
('American Beauty (1999)', 'Drama|Romance')
('Dances with Wolves (1990)', 'Adventure|Drama|Western')
('Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy')
('Apollo 13 (1995)'

检查Campaign中所使用的，指标最好的Solution的指标究竟是多少。
Personalize会自动将数据拆分出训练集和测试集，用于计算指标。

In [3]:
import boto3
from pprint import pprint

personalize = boto3.client('personalize')

response = personalize.get_solution_metrics(
    solutionVersionArn = \
    'arn:aws:personalize:us-west-2:188869792837:solution/personalize-recommender-demo2/f6fe7243'
)

pprint(response['metrics'])

{u'coverage': 0.0094,
 u'mean_reciprocal_rank_at_25': 0.0067,
 u'normalized_discounted_cumulative_gain_at_10': 0.0083,
 u'normalized_discounted_cumulative_gain_at_25': 0.0161,
 u'normalized_discounted_cumulative_gain_at_5': 0.0049,
 u'precision_at_10': 0.0016,
 u'precision_at_25': 0.0019,
 u'precision_at_5': 0.0013}
